In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions as f
from pyspark.sql import Window

spark = SparkSession.builder \
    .appName('DataFrame') \
    .master('local[*]') \
    .getOrCreate()

----

## Adult Data Set

https://archive.ics.uci.edu/ml/datasets/adult

In [8]:
col_names = ["age", "workclass", "fnlwgt", "education", "education-num","marital-status", "occupation", 
             "relationship", "race", "sex", "capital-gain", "capital-loss", "hours-per-week", 
             "native-country", "earnings"]

df = spark.read.csv("adult.data", header=False, inferSchema=True, ignoreLeadingWhiteSpace=True)

df = df.select(*[f.col(old).alias(new) for old, new in zip(df.columns, col_names)]).drop("fnlwgt").dropna("any")

df.select("age", "workclass", "education", "education-num","marital-status", "occupation").show(3)
df.select("relationship", "race", "sex", "capital-gain", "capital-loss", "hours-per-week").show(3)
df.select("native-country", "earnings").show(3)


+---+----------------+---------+-------------+------------------+-----------------+
|age|       workclass|education|education-num|    marital-status|       occupation|
+---+----------------+---------+-------------+------------------+-----------------+
| 39|       State-gov|Bachelors|           13|     Never-married|     Adm-clerical|
| 50|Self-emp-not-inc|Bachelors|           13|Married-civ-spouse|  Exec-managerial|
| 38|         Private|  HS-grad|            9|          Divorced|Handlers-cleaners|
+---+----------------+---------+-------------+------------------+-----------------+
only showing top 3 rows

+-------------+-----+----+------------+------------+--------------+
| relationship| race| sex|capital-gain|capital-loss|hours-per-week|
+-------------+-----+----+------------+------------+--------------+
|Not-in-family|White|Male|        2174|           0|            40|
|      Husband|White|Male|           0|           0|            13|
|Not-in-family|White|Male|           0|        

> **ZADANIA**:
1. Oblicz średnią `capital-gain` oraz `capital-loss` dla każdej narodowości - wynik uporządkuj malejąco po średnim zysku
2. Oblicz jaki procent kobiet zarabia ponad 50K w podziale na wykształcenie - wynik zaokrąglij do 2 miejsc po przecinku
3. Oblicz ile godzin średnio w ciągu tygodnia pracują osoby `Never-married` w stosunku do reszty społeczeństwa
4. W jakim wieku najczęściej zarabia sie powyżej 50K?
5. W podziale ze względu na zmienną `race` oblicz stosunek liczby kobiet do mężczyzn
6. Oblicz różnicę pomiędzy średnim czasem pracy osób uzyskujących `capital-gain` a resztą populacji
7. Oblicz róznicę wieku do średniego wieku osób z danym poziomem wykształcenia

In [14]:
#Oblicz średnią capital-gain oraz capital-loss dla każdej narodowości - wynik uporządkuj malejąco po średnim zysku,

df.groupBy("native-country").\
agg(f.avg("capital-gain").alias("avg_gain"),
   f.avg("capital-loss").alias("avg_loss")).\
orderBy("avg_gain", ascending=False).show()

+------------------+------------------+------------------+
|    native-country|          avg_gain|          avg_loss|
+------------------+------------------+------------------+
|             India|           3604.92|            131.12|
|             Japan|2546.2258064516127| 57.53225806451613|
|            Taiwan|2147.0392156862745| 154.8235294117647|
|              Iran|2044.6511627906978|126.81395348837209|
|             South|         1869.7375|           169.425|
|                 ?|1806.5180102915951|118.26586620926244|
|            Canada|1504.1322314049587|129.93388429752065|
|Dominican-Republic| 1476.057142857143| 25.17142857142857|
|            Greece| 1283.896551724138|295.44827586206895|
|     United-States|1089.2299280082277| 88.51059307507714|
|          Cambodia| 1027.842105263158|183.05263157894737|
|       Philippines|1018.3434343434343| 98.95454545454545|
|           Germany| 887.0948905109489| 77.97810218978103|
|           England| 864.7222222222222| 82.7555555555555

In [30]:
#Oblicz jaki procent kobiet zarabia ponad 50K w podziale na wykształcenie - wynik zaokrąglij do 2 miejsc po przecinku

df.filter(df.sex == "Female").\
withColumn('high_salary', f.when(f.col("earnings")!='<=50K', 1).\
          otherwise(0)).groupBy("education").\
agg(f.round( (f.sum(f.col("high_salary"))/f.count("high_salary")),2).\
alias("%")).orderBy("%").show()

+------------+----+
|   education|   %|
+------------+----+
|     1st-4th| 0.0|
|   Preschool| 0.0|
|        10th|0.01|
|     7th-8th|0.01|
|     5th-6th|0.02|
|        11th|0.02|
|         9th|0.03|
|        12th|0.03|
|Some-college|0.07|
|     HS-grad|0.07|
|  Assoc-acdm|0.13|
|   Assoc-voc|0.13|
|   Bachelors|0.21|
|     Masters|0.33|
| Prof-school|0.46|
|   Doctorate|0.58|
+------------+----+



In [48]:
#Oblicz ile godzin średnio w ciągu tygodnia pracują osoby Never-married w stosunku do reszty społeczeństwa
df.select('hours-per-week',
f.when(f.col('marital-status')=='Never-married','yes').\
         otherwise('no').alias('is_NM')).groupBy('is_NM').avg().show()

+-----+-------------------+
|is_NM|avg(hours-per-week)|
+-----+-------------------+
|   no|  42.14526007861779|
|  yes|   36.9399981278667|
+-----+-------------------+



In [53]:
#W jakim wieku najczęściej zarabia sie powyżej 50K?
df.select('age',f.when(f.col("earnings")!='<=50K', 1).\
          otherwise(0).alias('high_salary')).groupBy("age").\
agg(f.round( (f.sum(f.col("high_salary"))/f.count("high_salary")),2).\
alias('%')).orderBy('%', asceding=False).show()

+---+----+
|age|   %|
+---+----+
| 21| 0.0|
| 17| 0.0|
| 85| 0.0|
| 82| 0.0|
| 88| 0.0|
| 20| 0.0|
| 87| 0.0|
| 86| 0.0|
| 19| 0.0|
| 18| 0.0|
| 23|0.01|
| 22|0.02|
| 24|0.04|
| 25|0.06|
| 26|0.08|
| 80|0.09|
| 84| 0.1|
| 27| 0.1|
| 76|0.11|
| 72|0.13|
+---+----+
only showing top 20 rows



----

## SalaryHistory

In [ ]:
import random
random.seed(123)

years = 6
names = ['Alice', 'Betty', 'Chris', 'Dan', 'Greg']
names_count = len(names)
names = sorted(names*years)
year = [y for y in range(2005, 2005+years)] * names_count
starting_salary = [round(random.gauss(4000, 1000),2) for i in range(names_count)]
salary = [0 for i in range(years * names_count)]
salary[::years] = starting_salary
for n in range(names_count):
    for y in range(years-1):
        index = (years*n+1)+y
        salary[index] = round(salary[index-1]*(1+random.gauss(0.1,0.09)),2)

In [ ]:
salaryHistory = spark.createDataFrame([Row(name=n, year=y, salary=s) for n,y,s in zip(names, year, salary)])\
.filter((f.col('name') != 'Greg') | (f.col('year') != 2006))\
.union(spark.createDataFrame([Row('Alice', 3000, 2005)])).orderBy("name","year")

In [ ]:
salaryHistory.show(30)

> **ZADANIA**:
1. Oblicz ile razy powtarza się każde z imion w salaryHistory
2. Oblicz średnią, minimum i maksimum z pensji w każdym roku, lata posortuj malejąco, wyniki zaokrąglij do pełnych wartości
3. Oblicz różnicę między pensją w danym roku a średnią pensją danej osoby na przestrzeni wszystkich lat
4. Porównaj pensje ludzi pomiedzy najwcześniejszym i najpóźniejszym rokiem ich pracy
5. Czy komukolwiek obniżyła się pensja w stosunku do roku poprzedniego? **a.** Ile osób było kiedykolwiek w takiej sytuacji? **b.** Jaki jest rozkład częstości takich przypadków w zależności od roku?
6. Oblicz różnicę w pensjach w stosuku do: **a.** najwyższej pensji danej osoby, **b.** drugiej najwyższej pensji danej osoby
7. Oblicz medianę wszystkich zarobków
8. Oblicz medianę zarobków poszczególnych osób